In [32]:
import nltk
import numpy as np 
import pandas as pd 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval

In [33]:
data = pd.read_csv('data/Hotel_Reviews.csv')
data.shape

(515738, 17)

In [34]:
data['Hotel_Address'] = data.Hotel_Address.str.replace('United Kingdom', 'UK')
data['countries'] = data.Hotel_Address.apply(
    lambda x: x.split(' ')[-1]
)

data.countries.unique()

array(['Netherlands', 'UK', 'France', 'Spain', 'Italy', 'Austria'],
      dtype=object)

In [35]:
def impute(column):
    column = column[0]
    
    if(type(column) != list):
        return "".join(literal_eval(column))
    else:
        return column

In [36]:
data.Tags = data[['Tags']].apply(impute, axis=1)
data.shape

C:\Users\JOSHUA KYLE\AppData\Local\Temp\ipykernel_4280\2471498850.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  column = column[0]


(515738, 18)

In [37]:
data.sample(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,countries
475992,Via Gaetano Airaghi 125 20153 Milan Italy,229,7/14/2017,6.9,Idea Hotel Milano San Siro,Switzerland,If you want a good coffee for breakfast you s...,17,5742,Good location if you travel by car and need t...,17,2,8.3,Leisure trip Family with young children Dou...,20 days,45.472065,9.077787,Italy
428751,Rennweg 16 03 Landstra e 1030 Vienna Austria,370,4/12/2016,8.9,Austria Trend Hotel Savoyen Vienna,Germany,Horribly designed restaurant and bar,7,4016,Big rooms,3,10,6.7,Leisure trip Solo traveler Deluxe Room Sta...,478 day,48.195044,16.384475,Austria
157213,34 Norfolk Place Paddington Westminster Boroug...,634,8/8/2015,7.9,Norfolk Towers Paddington,New Zealand,No Negative,0,4065,The room was exactly what we booked very clea...,23,1,9.6,Leisure trip Couple Club Double Room Staye...,726 day,51.516288,-0.172108,UK


In [38]:
data.countries = data.countries.str.lower()
data.Tags = data.Tags.str.lower()

In [39]:
# def recommend_hotel(location, description):
#     description = description.lower()
#     word_tokenize(description)
#     stop_words = stopwords.words('english')
#     lemm = WordNetLemmatizer()
#     filtered = {word for word in description if not word in stop_words}
#     filtered_set = set()
    
#     for fs in filtered:
#         filtered_set.add(lemm.lemmatize(fs))
        
#     country = data[data.countries == location.lower()]
#     country = country.set_index(np.arange(country.shape[0]))
    
#     list1 = []
#     list2 = []
#     cos = []
    
#     for i in range(country.shape[0]):
#         temp_token = word_tokenize(country.Tags[i])
#         temp_set = [word for word in temp_token if not word in stop_words]
#         temp2_set = set()
        
#         for s in temp_set:
#             temp2_set.add(lemm.lemmatize(s))
            
#         vector = temp2_set.intersection(filtered_set)
#         cos.append(len(vector))
        
#     country.similarity = cos
#     country = country.sort_values(by='similarity', ascending=False)
#     country.drop_duplicates(subset='Hotel_Name', keep='first', inplace=True)
#     country.sort_values('Average_Score', ascending=False, inplace=True)
#     country.reset_index(inplace=True)
    
#     return country[['Hotel_Name', 'Average_Score', 'Hotel_Address']].head()

In [40]:
def recommend_hotel(location, description):
    description = description.lower()
    word_tokenize(description)
    stop_words = set(stopwords.words('english'))
    lemm = WordNetLemmatizer()
    filtered = {word for word in description if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))

    country = data[data['countries'] == location.lower()]
    country = country.set_index(np.arange(country.shape[0]))
    list1 = []; list2 = []; cos = [];
    for i in range(country.shape[0]):
        temp_token = word_tokenize(country['Tags'][i])
        temp_set = [word for word in temp_token if not word in stop_words]
        temp2_set = set()
        for s in temp_set:
            temp2_set.add(lemm.lemmatize(s))
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
    country['similarity'] = cos 
    country = country.sort_values(by='similarity', ascending=False)
    country.drop_duplicates(subset='Hotel_Name', keep='first', inplace=True)
    country.sort_values('Average_Score', ascending=False, inplace=True)
    country.reset_index(inplace=True)
    
    return country [["Hotel_Name", "Average_Score", "Hotel_Address"]].head()

In [41]:
data.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng', 'countries'],
      dtype='object')

In [42]:
recommend_hotel('UK', 'I am going on a honeymoon, I need a honeymoon suite room for 3 nights')

,Hotel_Name,Average_Score,Hotel_Address
0,Haymarket Hotel,9.6,1 Suffolk Place Westminster Borough London SW1...
1,41,9.6,41 Buckingham Palace Road Westminster Borough ...
2,Taj 51 Buckingham Gate Suites and Residences,9.5,Buckingham Gate Westminster Borough London SW1...
3,Charlotte Street Hotel,9.5,15 17 Charlotte Street Hotel Westminster Borou...
4,Ham Yard Hotel,9.5,One Ham Yard Westminster Borough London W1D 7D...
